In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
from cryptocompy import coin,price
import tqdm
import seaborn as sns
sns.set();

import io
import requests
import json
from datetime import datetime
import time
from time import gmtime, strftime

from sklearn import metrics
from fbprophet import Prophet
plt.rcParams["figure.figsize"] = [14,8]
pd.set_option('float_format', '{:f}'.format)

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import datetime
from datetime import datetime as dt
from datetime import timedelta


from tqdm import tqdm_notebook
from livelossplot.keras import PlotLossesCallback


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
import catboost
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU

from keras import optimizers
from keras import backend as K

adam = optimizers.Adam(lr=0.01, clipnorm=1.)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def nn_r2_score(y_true, y_pred):
    total_error = K.sum(K.square( y_true - K.mean(y_true) ) )
    residual_error = K.sum(K.square( y_true - y_pred ))
    R_squared = 1 - (residual_error / total_error)
    return -R_squared


from math import sin, cos

from gaft import GAEngine
from gaft.components import BinaryIndividual,DecimalIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection, LinearRankingSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput

# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

# Built-in best fitness analysis.
from gaft.analysis.fitness_store import FitnessStore

import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

Using TensorFlow backend.


In [2]:
df = pd.DataFrame()
d1 = pd.DataFrame()
d2 = pd.DataFrame()
d3 = pd.DataFrame()
d4 = pd.DataFrame()
d5 = pd.DataFrame()

d6 = pd.DataFrame()

with SSHTunnelForwarder(('206.189.186.74', 22), ssh_password='crypto1234', ssh_username='aagam', remote_bind_address=('127.0.0.1', 3306)) as server:
    conn = MySQLdb.connect(host='localhost', port=server.local_bind_port, user='ianbalina', passwd='5!sT3jt26K%tFN*W',db='ianbalina')
    cursor = conn.cursor()
    
    #cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
    #m = cursor.fetchone()
    query = '''SELECT * FROM ianbalina.ico_price_daily_summaries;'''
    data = pd.read_sql_query(query, conn)
    df = pd.concat([df,data])
    
    query = '''SELECT * FROM ianbalina.ico_grade_history;'''
    data = pd.read_sql_query(query, conn)
    d1 = pd.concat([d1,data])
    
    query = '''SELECT * FROM ianbalina.ico_analytics;'''
    data = pd.read_sql_query(query, conn)
    d2 = pd.concat([d2,data])
    
    query = '''SELECT * FROM ianbalina.icos;'''
    data = pd.read_sql_query(query, conn)
    d3 = pd.concat([d3,data])
    
    query = '''SELECT * FROM ianbalina.ico_analytics;'''
    data = pd.read_sql_query(query, conn)
    d4 = pd.concat([d4,data])
    
    query = '''SELECT * FROM ianbalina.ico_grade_history_scores;'''
    data = pd.read_sql_query(query, conn)
    d5 = pd.concat([d5,data])
    
    query = '''SELECT * FROM ianbalina.tech_scorecard_options;'''
    data = pd.read_sql_query(query, conn)
    d6 = pd.concat([d6,data])
    
    conn.close() 

I1013 08:54:36.120357  3112 transport.py:1819] Connected (version 2.0, client OpenSSH_7.6p1)
I1013 08:54:38.066727  3112 transport.py:1819] Authentication (password) successful!


In [3]:
def get_bins(v):
    if v <= 10:
        return 0
    elif v > 10 and v <= 20:
        return 1
    elif v > 20 and v <= 30:
        return 2
    elif v > 30 and v <= 40:
        return 3
    elif v > 40 and v <= 50:
        return 4
    elif v > 50 and v <= 60:
        return 5
    elif v > 60 and v <= 70:
        return 6
    elif v > 70 and v <= 80:
        return 7
    elif v > 80 and v <= 90:
        return 8
    elif v > 90:
        return 9

In [4]:
def get_technology_accuracy(weights, max_score, tech):
    t1 = pd.DataFrame()

    for i in tech['ico_symbol'].unique().tolist():
        temp = tech[tech['ico_symbol'] == i]
        temp = temp.sort_values('question_number')
        score = temp['score'] * weights[:len(temp)] 
        
        t1 = pd.concat([t1,pd.DataFrame([i, score.sum() / max_score * 100 / sum(weights[:len(temp)]) * len(temp)]).T])
        
    t1.columns = ['Coin','Score']
    #ids = tech['ico_id'][tech['ico_symbol'].isin(t1['Coin'])].unique()
    alpha = []
    for coin in t1['Coin']:
        k = tech['ico_id'][tech['ico_symbol'] == coin].unique()
        alpha.append(d4['alpha'][d4['ico_id'] == k[0]].values[0])
    
    t1['Alpha'] = alpha

    t1['Score_bins'] = t1['Score'].apply(get_bins)
    t1['Alpha_bins'] = pd.cut(t1['Alpha'],[np.floor(t1['Alpha'].min()),0,np.ceil(t1['Alpha'].max())])

    final = t1['Alpha_bins'].groupby([t1['Score_bins'],t1['Alpha_bins']]).count().unstack()

    x0 = []
    x1 = []
    for j in range(10):
        try:
            x0.append(final.loc[j,0])
        except:
            x0.append(0)

    for j in range(10):
        try:
            x1.append(final.loc[j,1])
        except:
            x1.append(0)

    x = pd.DataFrame()
    x[0] = x0
    x[1] = x1
    x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']

    x.fillna(0,inplace=True)

    y = x.sum(axis=1)
    y1 = (x.iloc[0:5,0] / y[:5]) 
    y2 = (x.iloc[5:,1] / y[5:])

    x['Accuracy'] = y1.append(y2).fillna(0).astype('float64').values * 100
    x.columns = [str(final.columns[0]),str(final.columns[1]),'Accuracy']

    accuracy = x.iloc[0:5,0].append(x.iloc[5:10,1]).sum() / x.iloc[:,:2].sum(axis=1).sum()
    return x, accuracy

In [5]:
tech = d5[d5['type'] == 2]
d1 = d1[d1['date'] == d1['date'].sort_values().iloc[-1]]
tech = tech[tech['ico_grade_history_id'].isin(d1['id'].unique())]
tech = pd.merge(tech,d1[['id','ico_id']], left_on = 'ico_grade_history_id', right_on = 'id', how='left')
tech = tech.drop('id_y', axis=1)
tech.rename(columns={'id_x':'id'}, inplace=True)

ids = d3['id'][d3['technology_published_at'].notnull()].unique().tolist()
coin = []
for id in ids:
    coin.append(df['ico_symbol'][df['ico_id'] == id].unique().tolist()[0])
    
tech = tech[tech['ico_id'].isin(ids)]
temp = pd.DataFrame([ids,coin]).T
temp.columns = ['ico_id','ico_symbol']

tech = pd.merge(tech,temp)
tech['question_number'] = tech['name'].str.split('_').str.get(-1)

In [6]:
weights = np.ones(28)
max_score = d6['score'].groupby([d6['tech_scorecard_question_id']]).max().sum()

get_technology_accuracy(weights, max_score, tech)

(          (-142.0, 0.0]  (0.0, 241.0]   Accuracy
 (0,10]         0.000000      0.000000   0.000000
 (10,20]        0.000000      0.000000   0.000000
 (20,30]        2.000000      0.000000 100.000000
 (30,40]        3.000000      1.000000  75.000000
 (40,50]        3.000000      0.000000 100.000000
 (50,60]        2.000000      2.000000  50.000000
 (60,70]        6.000000      4.000000  40.000000
 (70,80]        5.000000      5.000000  50.000000
 (80,90]        7.000000      2.000000  22.222222
 (90,100]       1.000000      2.000000  66.666667, 0.5111111111111111)

In [7]:
ids = d3['id'][d3['technology_published_at'].notnull()].unique().tolist()
coin = []
for id in ids:
    coin.append(df['ico_symbol'][df['ico_id'] == id].unique().tolist()[0])
    
temp = pd.DataFrame()
temp['Coin'] = coin
temp['Score'] = d1['tech_scorecard_score'][d1['ico_id'].isin(ids)].values

alpha = []
for coin in temp['Coin']:
    k = tech['ico_id'][tech['ico_symbol'] == coin].unique()
    alpha.append(d4['alpha'][d4['ico_id'] == k[0]].values[0])

temp['Alpha'] = alpha
temp[['Score','Alpha']] = temp[['Score','Alpha']].astype('float64')
'Correlation: ', temp.corr().iloc[0,1]

('Correlation: ', 0.06943535407610386)

In [8]:
temp

,Coin,Score,Alpha
0,ZRX,81.818200,-11.549000
1,TRX,81.818200,-0.806200
2,ONT,70.909100,-11.076700
3,ETH,98.181800,10.982000
4,NEO,85.454500,0.321500
5,BNB,90.909100,-12.474500
6,ADA,87.272700,-5.906000
7,LSK,74.545500,-6.055400
8,REP,74.545500,-0.239500
9,QTUM,67.272700,-2.874300


In [9]:
temp['Score_bins'] = pd.cut(temp['Score'],[0,10,20,30,40,50,60,70,80,90,100])
temp['Alpha_bins'] = pd.cut(temp['Alpha'],[np.floor(temp['Alpha'].min()),0,np.ceil(temp['Alpha'].max())])
final = temp['Alpha_bins'].groupby([temp['Score_bins'],temp['Alpha_bins']]).count().unstack()
final

Alpha_bins  (-142.0, 0.0]  (0.0, 241.0]
Score_bins                             
(20, 30]         2.000000           nan
(30, 40]         3.000000      1.000000
(40, 50]         3.000000           nan
(50, 60]         2.000000      2.000000
(60, 70]         6.000000      4.000000
(70, 80]         5.000000      5.000000
(80, 90]         7.000000      2.000000
(90, 100]        1.000000      2.000000

In [10]:
x = pd.DataFrame()
x[0] = [0,0] + final.values[:,0].tolist()
x[1] = [0,0] + final.values[:,1].tolist()
x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']

x.fillna(0,inplace=True)

y = x.sum(axis=1)
y1 = (x.iloc[0:5,0] / y[:5]) 
y2 = (x.iloc[5:,1] / y[5:])

x['Accuracy'] = y1.append(y2).fillna(0).astype('float64').values * 100
x.columns = [str(final.columns[0]),str(final.columns[1]),'Accuracy']
x

,"(-142.0, 0.0]","(0.0, 241.0]",Accuracy
"(0,10]",0.000000,0.000000,0.000000
"(10,20]",0.000000,0.000000,0.000000
"(20,30]",2.000000,0.000000,100.000000
"(30,40]",3.000000,1.000000,75.000000
"(40,50]",3.000000,0.000000,100.000000
"(50,60]",2.000000,2.000000,50.000000
"(60,70]",6.000000,4.000000,40.000000
"(70,80]",5.000000,5.000000,50.000000
"(80,90]",7.000000,2.000000,22.222222
"(90,100]",1.000000,2.000000,66.666667


In [11]:
x.iloc[0:5,0].append(x.iloc[5:10,1]).sum() / x.iloc[:,:2].sum(axis=1).sum()

0.5111111111111111

In [12]:
weights = [9, 2, 6, 4, 9, 5, 2, 2, 4, 10, 7, 3, 0, 4, 1, 3, 6, 10, 10, 10, 3, 0, 6, 1, 5, 9, 5, 10]

max_score = d6['score'].groupby([d6['tech_scorecard_question_id']]).max().sum()

y,z = get_technology_accuracy(weights, max_score, tech)

In [13]:
y

,"(-142.0, 0.0]","(0.0, 241.0]",Accuracy
"(0,10]",0.000000,0.000000,0.000000
"(10,20]",1.000000,0.000000,100.000000
"(20,30]",5.000000,1.000000,83.333333
"(30,40]",3.000000,1.000000,75.000000
"(40,50]",7.000000,3.000000,70.000000
"(50,60]",5.000000,6.000000,54.545455
"(60,70]",8.000000,3.000000,27.272727
"(70,80]",0.000000,2.000000,100.000000
"(80,90]",0.000000,0.000000,0.000000
"(90,100]",0.000000,0.000000,0.000000


In [12]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.7.3.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.24.2
pytest: None
pip: 19.1.1
setuptools: 41.0.1
Cython: 0.29.13
numpy: 1.16.4
scipy: 1.3.0
pyarrow: None
xarray: None
IPython: 7.6.1
sphinx: 2.1.2
patsy: None
dateutil: 2.8.0
pytz: 2019.1
blosc: None
bottleneck: None
tables: None
numexpr: None
feather: None
matplotlib: 3.1.1
openpyxl: 2.6.3
xlrd: 1.0.0
xlwt: None
xlsxwriter: None
lxml.etree: 4.4.0
bs4: 4.7.1
html5lib: None
sqlalchemy: None
pymysql: 0.9.3
psycopg2: None
jinja2: 2.10.1
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: 0.7.4
gcsfs: None


In [13]:
tech = d5[d5['type'] == 2]

In [14]:
d1 = d1[d1['date'] == d1['date'].sort_values().iloc[-1]]

In [15]:
tech = tech[tech['ico_grade_history_id'].isin(d1['id'].unique())]

In [17]:
tech = pd.merge(tech,d1[['id','ico_id']], left_on = 'ico_grade_history_id', right_on = 'id', how='left')

In [18]:
tech = tech.drop('id_y', axis=1)

In [19]:
tech.rename(columns={'id_x':'id'}, inplace=True)

In [20]:
ids = d3['id'][d3['technology_published_at'].notnull()].unique().tolist()
coin = []
for id in ids:
    coin.append(df['ico_symbol'][df['ico_id'] == id].unique().tolist()[0])

In [21]:
tech = tech[tech['ico_id'].isin(ids)]

In [22]:
temp = pd.DataFrame([ids,coin]).T
temp.columns = ['ico_id','ico_symbol']

In [23]:
tech = pd.merge(tech,temp)

In [24]:
tech['question_number'] = tech['name'].str.split('_').str.get(-1)

In [32]:
d6['score'].groupby([d6['tech_scorecard_question_id']]).max().sum()

55.0

In [106]:
def get_bins(v):
    if v <= 10:
        return 0
    elif v > 10 and v <= 20:
        return 1
    elif v > 20 and v <= 30:
        return 2
    elif v > 30 and v <= 40:
        return 3
    elif v > 40 and v <= 50:
        return 4
    elif v > 50 and v <= 60:
        return 5
    elif v > 60 and v <= 70:
        return 6
    elif v > 70 and v <= 80:
        return 7
    elif v > 80 and v <= 90:
        return 8
    elif v > 90:
        return 9

In [141]:
def get_technology_accuracy(weights, max_score, tech):
    t1 = pd.DataFrame()

    for i in tech['ico_symbol'].unique().tolist():
        temp = tech[tech['ico_symbol'] == i]
        temp = temp.sort_values('question_number')
        score = temp['score'] * weights[:len(temp)] 
        
        t1 = pd.concat([t1,pd.DataFrame([i, score.sum() / max_score * 100 / sum(weights[:len(temp)]) * len(temp)]).T])
        
    t1.columns = ['Coin','Score']
    #ids = tech['ico_id'][tech['ico_symbol'].isin(t1['Coin'])].unique()
    alpha = []
    for coin in t1['Coin']:
        k = tech['ico_id'][tech['ico_symbol'] == coin].unique()
        alpha.append(d4['alpha'][d4['ico_id'] == k[0]].values[0])
    
    t1['Alpha'] = alpha

    t1['Score_bins'] = t1['Score'].apply(get_bins)
    t1['Alpha_bins'] = pd.cut(t1['Alpha'],[np.floor(t1['Alpha'].min()),0,np.ceil(t1['Alpha'].max())])

    final = t1['Alpha_bins'].groupby([t1['Score_bins'],t1['Alpha_bins']]).count().unstack()

    x0 = []
    x1 = []
    for j in range(10):
        try:
            x0.append(final.loc[j,0])
        except:
            x0.append(0)

    for j in range(10):
        try:
            x1.append(final.loc[j,1])
        except:
            x1.append(0)

    x = pd.DataFrame()
    x[0] = x0
    x[1] = x1
    x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']

    x.fillna(0,inplace=True)

    y = x.sum(axis=1)
    y1 = (x.iloc[0:5,0] / y[:5]) 
    y2 = (x.iloc[5:,1] / y[5:])

    x['Accuracy'] = y1.append(y2).fillna(0).astype('float64').values * 100
    x.columns = [str(final.columns[0]),str(final.columns[1]),'Accuracy']

    accuracy = x.iloc[0:5,0].append(x.iloc[5:10,1]).sum() / x.iloc[:,:2].sum(axis=1).sum()
    return x, accuracy

In [13]:
weights = np.ones(28)*0.1
max_score = d6['score'].groupby([d6['tech_scorecard_question_id']]).max().sum()

get_technology_accuracy(weights, max_score, tech)

(          (-143.0, 0.0]  (0.0, 242.0]   Accuracy
 (0,10]         0.000000      0.000000   0.000000
 (10,20]        0.000000      0.000000   0.000000
 (20,30]        2.000000      0.000000 100.000000
 (30,40]        3.000000      1.000000  75.000000
 (40,50]        3.000000      0.000000 100.000000
 (50,60]        2.000000      2.000000  50.000000
 (60,70]        6.000000      4.000000  40.000000
 (70,80]        5.000000      5.000000  50.000000
 (80,90]        7.000000      2.000000  22.222222
 (90,100]       1.000000      2.000000  66.666667, 0.5111111111111111)

In [16]:
indv_template = DecimalIndividual(ranges=[(0,10)]*28, eps=[1]*28)
population = Population(indv_template=indv_template, size=100).init()

selection = LinearRankingSelection()
crossover = UniformCrossover(pc=0.8, pe=0.5)
mutation = FlipBitMutation(pm=0.1)

# Create genetic algorithm engine.
engine = GAEngine(population=population, selection=selection,
                  crossover=crossover, mutation=mutation,
                  analysis=[FitnessStore])

@engine.fitness_register
#@engine.minimize
def fitness(indv):
    w = list(indv.solution)
    _, res = get_technology_accuracy(w, max_score, tech)
    res = float(abs(res))
    #print(w,': ',res)
    return res

def get_test_results(w1,w2,w3,w4):
    predictions = m1*w1 + m2*w2 + m3*w3 + m4*w4
    print('\nRMSE error: ', np.sqrt(metrics.mean_squared_error(Y_test, predictions)))
    for i,prediction in enumerate(predictions[:15]):
        print ('Target: %s, Predicted: %s' % (Y_test.tolist()[i], abs(prediction)))

# Define on-the-fly analysis.
@engine.analysis_register
class ConsoleOutputAnalysis(OnTheFlyAnalysis):
    interval = 1
    master_only = True

    def register_step(self, g, population, engine):
        best_indv = population.best_indv(engine.fitness)
        msg = 'Generation: {}, best fitness: {:.3f}, best_individual: {}'.format(g, engine.ori_fmax, best_indv.solution)
        self.logger.info(msg)

    def finalize(self, population, engine):
        best_indv = population.best_indv(engine.fitness)
        x = best_indv.solution
        y = engine.fmax
        msg = 'Optimal solution: ({}, {})'.format(x, y)
        self.logger.info(msg)

if '__main__' == __name__:
    # Run the GA engine.
    engine.run(ng=100)
    best_indv = engine.population.best_indv(engine.fitness)
    w1,w2,w3,w4 = best_indv.solution
    get_test_results(w1,w2,w3,w4)

gaft.ConsoleOutputAnalysis   INFO     Generation: 0, best fitness: 0.556, best_individual: [5, 4, 7, 2, 8, 7, 0, 6, 2, 4, 9, 7, 5, 5, 7, 5, 2, 10, 6, 5, 9, 10, 3, 6, 4, 3, 4, 1]


I1010 08:48:07.707921 32656 <ipython-input-16-46d7ad2432ef>:37] Generation: 0, best fitness: 0.556, best_individual: [5, 4, 7, 2, 8, 7, 0, 6, 2, 4, 9, 7, 5, 5, 7, 5, 2, 10, 6, 5, 9, 10, 3, 6, 4, 3, 4, 1]


gaft.ConsoleOutputAnalysis   INFO     Generation: 1, best fitness: 0.578, best_individual: [2, 1, 9, 1, 6, 3, 2, 5, 1, 9, 0, 7, 3, 0, 8, 4, 5, 10, 0, 3, 6, 10, 8, 7, 5, 6, 9, 9]


I1010 08:48:50.848676 32656 <ipython-input-16-46d7ad2432ef>:37] Generation: 1, best fitness: 0.578, best_individual: [2, 1, 9, 1, 6, 3, 2, 5, 1, 9, 0, 7, 3, 0, 8, 4, 5, 10, 0, 3, 6, 10, 8, 7, 5, 6, 9, 9]


gaft.ConsoleOutputAnalysis   INFO     Generation: 2, best fitness: 0.578, best_individual: [2, 1, 9, 1, 6, 3, 2, 5, 1, 9, 0, 7, 3, 0, 8, 4, 5, 10, 0, 3, 6, 10, 8, 7, 5, 6, 9, 9]


...

gaft.FitnessStore   INFO     Best fitness values are written to best_fit.py


I1010 10:00:57.772454 32656 fitness_store.py:48] Best fitness values are written to best_fit.py


gaft.ConsoleOutputAnalysis   INFO     Optimal solution: ([9, 2, 6, 4, 9, 5, 2, 2, 4, 10, 7, 3, 0, 4, 1, 3, 6, 10, 10, 10, 3, 0, 6, 1, 5, 9, 5, 10], 0.6)


I1010 10:00:57.776458 32656 <ipython-input-16-46d7ad2432ef>:44] Optimal solution: ([9, 2, 6, 4, 9, 5, 2, 2, 4, 10, 7, 3, 0, 4, 1, 3, 6, 10, 10, 10, 3, 0, 6, 1, 5, 9, 5, 10], 0.6)


ValueError: too many values to unpack (expected 4)

In [ ]:
t2

In [131]:
weights = np.ones(28)*2
max_score = d6['score'].groupby([d6['tech_scorecard_question_id']]).max().sum()

t1 = pd.DataFrame()
for i in tech['ico_symbol'].unique().tolist():
    temp = tech[tech['ico_symbol'] == i]
    temp = temp.sort_values('question_number')
    score = temp['score'] * weights[:len(temp)] 
    t1 = pd.concat([t1,pd.DataFrame([i, score.sum() / max_score * 100 / sum(weights[:len(temp)]) * len(temp)]).T])

In [132]:
t1.columns = ['Coin','Score']

In [133]:
ids = tech['ico_id'][tech['ico_symbol'].isin(t1['Coin'])].unique()

In [134]:
t1['Alpha'] = d4['alpha'][d4['ico_id'].isin(ids)].values

In [135]:
t1['Score_bins'] = t1['Score'].apply(get_bins)
t1['Alpha_bins'] = pd.cut(t1['Alpha'],[np.floor(t1['Alpha'].min()),0,np.ceil(t1['Alpha'].max())])

In [136]:
t1

,Coin,Score,Alpha,Score_bins,Alpha_bins
0,ZRX,81.818182,-12.731300,8,"(-143.0, 0.0]"
0,TRX,81.818182,23.428200,8,"(0.0, 243.0]"
0,ONT,70.909091,-1.796800,7,"(-143.0, 0.0]"
0,ETH,98.181818,6.383400,9,"(0.0, 243.0]"
0,NEO,85.454545,-9.081200,8,"(-143.0, 0.0]"
0,BNB,90.909091,-41.463300,9,"(-143.0, 0.0]"
0,ADA,87.272727,-5.844000,8,"(-143.0, 0.0]"
0,LSK,74.545455,15.519000,7,"(0.0, 243.0]"
0,REP,74.545455,-11.537000,7,"(-143.0, 0.0]"
0,QTUM,67.272727,-0.737900,6,"(-143.0, 0.0]"


In [109]:
final = t1['Alpha_bins'].groupby([t1['Score_bins'],t1['Alpha_bins']]).count().unstack()
final

Alpha_bins  (-143.0, 0.0]  (0.0, 243.0]
Score_bins                             
2                1.000000      1.000000
3                2.000000      2.000000
4                2.000000      1.000000
5                4.000000           nan
6                3.000000      7.000000
7                8.000000      2.000000
8                6.000000      3.000000
9                2.000000      1.000000

In [121]:
x0 = []
x1 = []
for j in range(10):
    try:
        x0.append(final.loc[j,0])
    except:
        x0.append(0)
        
for j in range(10):
    try:
        x1.append(final.loc[j,1])
    except:
        x1.append(0)

In [124]:
x = pd.DataFrame()
x[0] = x0
x[1] = x1
x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']

x.fillna(0,inplace=True)

y = x.sum(axis=1)
y1 = (x.iloc[0:5,0] / y[:5]) 
y2 = (x.iloc[5:,1] / y[5:])

x['Accuracy'] = y1.append(y2).fillna(0).astype('float64').values * 100
x.columns = [str(final.columns[0]),str(final.columns[1]),'Accuracy']

In [125]:
x

,"(-143.0, 0.0]","(0.0, 243.0]",Accuracy
"(0,10]",0.000000,0.000000,0.000000
"(10,20]",0.000000,0.000000,0.000000
"(20,30]",1.000000,1.000000,50.000000
"(30,40]",2.000000,2.000000,50.000000
"(40,50]",2.000000,1.000000,66.666667
"(50,60]",4.000000,0.000000,0.000000
"(60,70]",3.000000,7.000000,70.000000
"(70,80]",8.000000,2.000000,20.000000
"(80,90]",6.000000,3.000000,33.333333
"(90,100]",2.000000,1.000000,33.333333


In [126]:
x.iloc[0:5,0].append(x.iloc[5:10,1]).sum() / x.iloc[:,:2].sum(axis=1).sum()

0.4